In [1]:
# D Duncan 18/01/19

# Setting up a synthetic surface scene to gauge the information content
#  from 2dvar surface retrievals with the full beam pattern. In other words,
#  create a synthetic surface scene with some overlying atmosphere, defined
#  on some grid, then see if a retrieval can discern perturbations in the
#  surface.
#
# Code originally copied from am2_findcal.ipynb

# Set up ARTS/Python environment


In [3]:
%env ARTS_INCLUDE_PATH=/home/dudavid/arts/controlfiles/
%env ARTS_DATA_PATH=/home/dudavid/arts/arts-xml/
%env ARTS_BUILD_PATH=/home/dudavid/arts/build/

%matplotlib inline
import glob
from h5py import File
from netCDF4 import Dataset

from typhon.arts.workspace import Workspace, arts_agenda
ws = Workspace(verbosity=0)
ws.execute_controlfile("general/general.arts")
ws.execute_controlfile("general/continua.arts")
ws.execute_controlfile("general/agendas.arts")
ws.execute_controlfile("general/planet_earth.arts")

env: ARTS_INCLUDE_PATH=/home/dudavid/arts/controlfiles/
env: ARTS_DATA_PATH=/home/dudavid/arts/arts-xml/
env: ARTS_BUILD_PATH=/home/dudavid/arts/build/
Loading ARTS API from: /home/dudavid/arts/build/src/libarts_api.so


In [4]:
from typhon.arts.workspace.variables import *

ws.Copy( ws.abs_xsec_agenda, ws.abs_xsec_agenda__noCIA )

ws.Copy( ws.iy_main_agenda, ws.iy_main_agenda__Emission )
# used to debug with sensor position and things:
#@arts_agenda
#def iy_main_agendaPY(ws):
#    ws.ppathCalc()
#    ws.iyEmissionStandard()
#    ws.Print(ws.ppath,0)
#ws.Copy( ws.iy_main_agenda, iy_main_agendaPY)
    
ws.Copy( ws.iy_space_agenda, ws.iy_space_agenda__CosmicBackground )
ws.Copy( ws.propmat_clearsky_agenda, ws.propmat_clearsky_agenda__OnTheFly )
ws.Copy( ws.ppath_agenda, ws.ppath_agenda__FollowSensorLosPath )
ws.Copy( ws.ppath_step_agenda, ws.ppath_step_agenda__GeometricPath )

In [5]:
# define absorbing species and sensor (here using metmm library, used again below)
ws.abs_speciesSet(species=["H2O-PWR98","O2-PWR93","N2-SelfContStandardType"])
ws.abs_lines_per_speciesSetEmpty()

# General Settings

In [6]:
ws.stokes_dim = 2 # to get V and H pol out of metmm
ws.iy_unit = "PlanckBT" # equivalent: ws.StringSet( iy_unit, "PlanckBT" )

# Atmosphere set up # (take Psfc, T from model later)

In [7]:
from typhon.physics.atmosphere import integrate_water_vapor
ws.atmosphere_dim = 1  # for 1DVAR
p = np.array([1000.,950.,900.,850.,800.,700.,600.,500.,300.,100.])*100.0  # keep it coarse
#p = np.array([1000.,975.,950.,925.,900.,850.,800.,750.,700.,650.,600.,550.,500.,400.,300.,200.,100.])*100.0
print(len(p))
ws.p_grid = p[:] #0.5 * (p[1:] + p[:-1])
print(ws.p_grid.value)
ws.AtmRawRead( basename = "planets/Earth/Fascod/tropical/tropical") #tropical atmosphere assumed
# WV given as VMR
ws.AtmosphereSet1D()
ws.AtmFieldsCalc()

ws.z_surface = np.asarray(ws.z_field)[0]
ws.t_surface = np.asarray(ws.t_field)[0]
trop_ss, trop_t = np.copy(ws.t_surface.value), np.copy(ws.t_field.value[:,0,0])
z = ws.z_field.value[:, 0, 0]

factor = 28.966/18.016 * 0.001 # convert WV mixing ratio (g/kg) to vmr (mol/mol)
trop_vmr = np.copy(ws.vmr_field.value[0,:,0,0])
#wvc = integrate_water_vapor(ws.vmr_field.value[0, :, 0, 0], ws.p_grid.value)
#print(wvc)

17
[100000.  97500.  95000.  92500.  90000.  85000.  80000.  75000.  70000.
  65000.  60000.  55000.  50000.  40000.  30000.  20000.  10000.]
trop vmr: [2.52196149e-02 2.37858298e-02 2.23147994e-02 2.08045368e-02
 1.93452537e-02 1.72987546e-02 1.49967833e-02 1.13252834e-02
 7.88189193e-03 5.34901910e-03 3.97190372e-03 3.19044020e-03
 2.26046337e-03 9.87003039e-04 2.65972393e-04 2.14005006e-05
 2.94033094e-06] ['2.94E-6', '2.52E-2', '1.05E-2', '9.06E-3']
[2500.0, 2500.0, 2500.0, 2500.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 10000.0, 10000.0, 10000.0, 10000.0]
39.631092634692116


# Sensor definition -- AMSR2 sensor setup w/ metmm

In [8]:
ws.ArrayOfIndexCreate("channels")  # ws.channels is set down below with metmm call

# initial sensor setup -- these are shorthand for available AMSR2 channels (no b-scan)
ch_str = ['6V','6H','7V','7H','10V','10H','18V','18H','23V','23H','36V','36H','89V','89H']

#nlo = 14
#csub = [[x for x in range(nlo)], [x+nlo for x in range(2)]] # 16 AMSR2 channels total
#csub = [[0,1,2,3,4,5,6,7,8,9,10,11,12,13],[]] # so 6 to 89GHz (no b-scan)
csub = [[0,1,2,3,4,5],[]] # so 6.9V/H, 7.3V/H, 10V/H  -- all sfc channels!
nch = np.size(np.ravel(csub[0]+csub[1]))
print('# channels using: ',nch)

ws.ArrayOfIndexCreate("viewing_angles") # necessary if just using one pencil beam angle?
#ws.viewing_angles = [1] # index of viewing angles in metmm file -- for AMSR2 only B-scan is different
# current setup is low freqs at index 1, high freqs at index 0

ws.sensor_pos  = np.array([[699.6e3]]) # altitude [taken from h5 header BELOW!!!]
#can take exact alt from file if desired, same with lat/lon values (does that have effect?)
ws.sensor_time = np.array([0.0]) # same here
ws.sensor_los  = np.array([[180.0]]) # since angle taken care of in metmm file 
# zenith angle followed by azimuth. zenith=0 is straight up, 180 is straight down. azimuth=0 is N, positive to east 

ws.IndexCreate("met_mm_accuracy") 
ws.met_mm_accuracy = 2 # points within each bandwidth to comute abs characteristics


# channels using:  14


# Perform some checks

In [9]:
ws.atmfields_checkedCalc( bad_partition_functions_ok = 1 )
ws.abs_xsec_agenda_checkedCalc()
ws.propmat_clearsky_agenda_checkedCalc()
ws.atmgeom_checkedCalc()
#ws.abs_lookupSetup() # not currently using lookup tables (on the fly instead)
#ws.abs_lookupCalc()

In [14]:
epaf = '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/EC/wv1609/'

#import cdsapi
#era5client = cdsapi.Client()
#
#era5client.retrieve(
#    'reanalysis-era5-single-levels',
#    {
#        'product_type':'reanalysis',
#        'format':'netcdf',
#        'variable':[
#            '10m_u_component_of_wind','10m_v_component_of_wind','10m_wind_direction',
#            '2m_dewpoint_temperature','2m_temperature','sea_surface_temperature',
#            'mean_sea_level_pressure'
#        ],
#        'year':'2016',
#        'month':'09',
#        'day':'21',
#        'time':'15:00'
#    },
#    'sfc_vars_210916_15Z.nc')

# read in T and q from ERA5:
era5p = epaf+'testy.nc'
era5s = epaf+'sfc_vars_210916_15Z.nc'
# fields are [time, levels, lat, lon] , so [1,17,721,1440] the way it was downloaded
# this file is 15Z, near the equator crossing time of the same 210916 orbit

era = Dataset(era5p)
era_q = era['q'][0,:,:,:]
era_mr = era_q / (1-era_q)
era_t = era['t'][0,:,:,:]
print(np.shape(era_t))
elo=era['longitude'][:]
ela=era['latitude'][:]
# shift grids from 0-360 lon to -180-180 lons?

eras = Dataset(era5s)
era_wi = ( eras['u10'][0,:,:]**2 + eras['v10'][0,:,:]**2 )**0.5  # get wind speed from vectors [m/s]
print(shinfo(era_wi))
era_wd = eras['dwi'][0,:,:]  # direction of 10m wind [degrees from N]
#era_td = eras['d2m'][:]  # 2m dewpoint temperature [K]   --- not being used right now!
era_ts = eras['t2m'][0,:,:]  # 2m air temperature [K]
print(shinfo(era_ts))
era_sp = eras['msl'][0,:,:]  # mean sea level pressure [Pa]
era_ss = eras['sst'][0,:,:]  # SST [K]
#print(info(era_wd[era_wd>0]))
era_wd[era_wd<0] = 0.0   # for missing values, set to 0 (N)

#grdmap( era_wd,  0, -90, 0, 360, 0)

(17, 721, 1440)
['7.90E-3', '2.69E+1', '6.24E+0', '3.72E+0', (721, 1440)]
['2.06E+2', '3.18E+2', '2.80E+2', '2.18E+1', (721, 1440)]


# Designate surface vars

In [15]:

#ws.NumericCreate("wind_speed")
#ws.NumericCreate("wind_direction")
#ws.wind_speed = 3.9
#ws.surface_skin_t = 290.2
#ws.wind_direction = 0.0 # can adjust based on analysis data later  -- default should be 0
# -- need sensor azimuth angle, and spacecraft heading, as arts azimuth is relative to N
#ws.salinity = .035 # default is .035 anyway

# transmittance only needed for running fastem:
#ws.VectorCreate("transmittance")
#print(ws.f_grid.value)
#ws.transmittance  = np.ones(ws.f_grid.value.shape) # or ones, or something else?
#print('wind speed, SST: ',ws.wind_speed.value,ws.surface_skin_t.value)#ws.t_surface.value.ravel()[0])

# add surface variables for jacobian calculation -- need to keep order consistent between snames/sdata
snames = ["Water skin temperature", "Wind speed", "Salinity", "Wind direction"]
sdata = np.array([299, 3.4, 0.035, 0]).reshape(4,1,1)
#sdata = np.array([ws.surface_skin_t.value, ws.wind_speed.value, 0.035, ws.wind_direction.value]).reshape(4,1,1)
#sdata = np.array([ws.t_surface.value.ravel()[0], ws.wind_speed.value, 0.035]).reshape(3,1,1)

# don't copy to arts variables yet... otherwise ycalc bombs due to SurfaceDummy [old issue]
ws.Copy(ws.surface_props_names, snames)
ws.Copy(ws.surface_props_data, sdata)


nrvar = 2 # num retrieved variables
rvars = np.zeros(nrvar)
rvars[:] = [4.2, 3.9] # corresponding to variable order set above

# set Xa (formally set below)
xa = np.zeros(nrvar) # if supplying user-defined a priori vector
#xa[:] = [ws.surface_skin_t.value, ws.wind_speed.value] # just perturbations from values defined above

# define jacobians, run ycalc 

In [16]:

ws.ArrayOfIndexCreate("viewing_angles_1")
ws.viewing_angles_1 = [0] # index, defined in metmm file
ws.ArrayOfIndexCreate("channels_1") # low freq channels EIA
ws.channels_1 = csub[0]
ws.Copy(ws.viewing_angles, ws.viewing_angles_1)
ws.Copy(ws.channels, ws.channels_1)

# re-execution of metmm control files ( since channels subset and angle have changed)
ws.execute_controlfile("instruments/metmm/sensor_descriptions/prepare_metmm.arts")
ws.execute_controlfile("instruments/metmm/sensor_descriptions/sensor_amsr2.arts") #atms.arts")
ws.execute_controlfile("instruments/metmm/sensor_descriptions/apply_metmm.arts") # to execute CF?

# NO NEED TO RUN JACOBIAN INIT IF USING RETRIEVAL DEF INIT
#ws.jacobianInit()  # initialize jacobian quantities, then add variables

In [17]:
# if using tessem... read tessem ascii files into arts memory
tez = 'F' # fastem is default
if use_tes:
    ws.TessemNNReadAscii(tessem_netv, "testdata/tessem_sav_net_V.txt")
    ws.TessemNNReadAscii(tessem_neth, "testdata/tessem_sav_net_H.txt") 
    tez = 'T'

# if using surfaceFastem, need to provide transmittances and wind direction:
ws.VectorCreate( "transmittance" )
print(ws.f_grid.value)
ws.transmittance = np.ones( ws.f_grid.value.shape )

if use_tes:
    @arts_agenda
    def iy_surface_agendaPY(ws):
        ws.SurfaceTessem()
        ws.iySurfaceRtpropCalc()
else:
    @arts_agenda
    def iy_surface_agendaPY(ws):
        ws.SurfaceFastem( transmittance = ws.transmittance ) 
        ws.iySurfaceRtpropCalc()
    
ws.Copy(ws.iy_surface_agenda, iy_surface_agendaPY) # copy python-defined agenda to ARTS

[6.8375e+09 7.0125e+09 7.2125e+09 7.3875e+09 1.0625e+10 1.0675e+10
 1.8650e+10 1.8750e+10 2.3700e+10 2.3900e+10 3.6250e+10 3.6750e+10
 8.8000e+10 8.9000e+10 9.0000e+10]


In [18]:
ws.SurfaceFastem


*-------------------------------------------------------------------*
Workspace method = SurfaceFastem
---------------------------------------------------------------------

FASTEM sea surface microwave emissivity parametrization.

The variable *surface_props_data* must contain these data:
  "Water skin temperature"
  "Wind speed"
  "Wind direction"
  "Salinity"

For some details and comments see *FastemStandAlone* and *surfaceFastem*.


Synopsis:

SurfaceFastem( surface_los, surface_rmatrix, 
               dsurface_rmatrix_dx, surface_emission, 
               dsurface_emission_dx, stokes_dim, atmosphere_dim, 
               lat_grid, lon_grid, f_grid, rtp_pos, rtp_los, 
               surface_props_data, surface_props_names, 
               dsurface_names, jacobian_do, transmittance, 
               fastem_version )


Authors: Patrick Eriksson


Variables:

OUT   surface_los (Matrix): 
      Downwelling radiation directions to consider in surface reflection.
OUT   surface_rmatrix (

In [ ]:
# will leave calls for retrieval definition setup, don't seem to be any harm...

ws.retrievalDefInit()  # initialize jacobian quantities, then add variables
sx_cov = np.diag(rvars) # CHANGE TO MATCH SIZE OF Xa!!
ws.retrievalAddSurfaceQuantity(
    g1=ws.lat_grid, g2=ws.lon_grid, quantity=snames[0])
ws.covmat_sxAddBlock(block = sx_cov[:1, :1])

ws.retrievalAddSurfaceQuantity(
    g1=ws.lat_grid, g2=ws.lon_grid, quantity=snames[1])
ws.covmat_sxAddBlock(block = sx_cov[1:, 1:])
ws.Copy(ws.xa, xa)

## set channel errors, currently just constant across channels
chan_err = 0.6 # for now all the same, in K

se_cov = np.diag(np.zeros(nch) + chan_err**2 ) # set channel assumed errors
ws.covmat_seSet(se_cov)
ws.retrievalDefClose()

ws.cloudboxOff()
ws.cloudbox_checkedCalc()
ws.sensor_checkedCalc()

ws.yCalc()  # calculate yf and jacobian matching x
print(ws.y.value)
am_tb = ws.y.value


[172.15586386  81.94570759 172.60580143  82.45536439 176.9174824
  87.86840997 204.16479839 131.65065171 239.51503674 194.66211674
 223.84439735 157.96578137 273.96166727 249.26243811]


In [ ]:
# now, loop over 'good' cloud-free pixels and compare ARTS-generated Tb to L1R

tb_sim, tb_obs = [], []
lo_sav, la_sav = [], []
tp_sav, wi_sav = [], []
ss_sav, sc_sav = [],[]
chcut, lwcut = 0.30, 8

for sc in range(nscans):#[600:621]:
    if np.mod(sc,150) == 0: print(sc)
    for px in range(npix)[30:210: 1]:  # avoiding scan edges for now!
        
        if tp[px,sc]>0 and ch[px,sc]<chcut and lw[px,sc]<lwcut: # dont bother with cloudy or bad retrievals
            
            # define era5 indices:
            if lo[px,sc] >= 0: elo_dex = int(np.floor(lo[px,sc]*4.0))
            if lo[px,sc] <  0: elo_dex = int(np.floor((lo[px,sc]+360)*4.0))
            ela_dex = int(np.floor((90-la[px,sc])*4.0)) # era5 goes from +90N to 90S
            
            # use ERA5 data for all (T, wind dir, bottom of P grid)
            ws.t_field.value[:,0,0] = era_t[:,ela_dex,elo_dex][::-1] 
            ws.t_field.value[0,0,0] = era_ts[ela_dex,elo_dex] # take 2m T for lowest level
            dwi = era_wd[ela_dex,elo_dex] # use era5 wind direction for all too
            #ws.p_grid.value[0] = era_sp[ela_dex,elo_dex]
            
            # regrid the vertical fields (as lowest value in p_grid changes) then run ARTS forward
            ws.AtmFieldsCalc()
            
            if use_era:
                ws.vmr_field.value[0,:,0,0] = 1e3*era_mr[::-1,ela_dex,elo_dex]*factor # MR to VMR
                # should use era5 dewpoint to fix lowest MR level here:
                wnd = era_wi[ela_dex,elo_dex]
                sst = era_ss[ela_dex,elo_dex]
                if(sst < 272 or wnd<0): print('low/bad era5 data')
            else:
                veemr = trop_vmr * tp[px,sc] / wvc    # scale input tropical profile to 1dvar result
                ws.vmr_field.value[0,:,0,0] = veemr
                wnd = wi[px,sc]
                sst = ss[px,sc]
            
            sdata = np.array([sst, wnd, 0.035, dwi ]).reshape(4,1,1)
            ws.Copy(ws.surface_props_data, sdata)
            
            ws.yCalc()
            
            # save tbs and other vars for further analysis
            tb_sim.append( np.copy(ws.y.value) )  #otherwise it's wrong as arts memory changes!
            tb_obs.append( tb_cal[px,sc,:] ) # use calibrated ones here?
            lo_sav.extend( [lo[px,sc]])
            la_sav.extend( [la[px,sc]] )
            tpw = integrate_water_vapor(np.copy(ws.vmr_field.value[0,:,0,0]), np.copy(ws.p_grid.value))
            tp_sav.extend( [tpw] )
            wi_sav.extend( [wnd] )
            ss_sav.extend( [sst] )
            sc_sav.extend( [px] ) # just scan position, aka pixel number in scan!
            
ncase = np.shape(tb_sim)[0]
print(ncase)
tb_sim = np.stack(tb_sim)
tb_obs = np.array(tb_obs) #.reshape(nch,ncase)
print('example point, first pixel:')
print(tb_sim[0,:])
print(tb_obs[0,:])
#print('TPW info: ',info(tp_sav))
#print(info(la_sav),info(lo_sav))


0
150
300
450
600
750
900
1050
1200
1350


In [ ]:
plt.style.use('bmh')

tbd = tb_obs - tb_sim

k=0
print(tbd[k,:], tp_sav[k],wi_sav[k],ss_sav[k])
k=2000
print(tbd[k,:], tp_sav[k],wi_sav[k],ss_sav[k])
k=3000
print(tbd[k,:], tp_sav[k],wi_sav[k],ss_sav[k])
k=1000
print(tbd[k,:], tp_sav[k],wi_sav[k],ss_sav[k])
#low = np.where(np.array(tp_sav)<14)  # testing
hx = np.arange(-12,12,.1)
p1 = plt.figure(figsize=[14,9])
outs = np.zeros([nch,2])
for c in range(nch)[:-2]:
    outs[c,:] = [round(np.mean(tbd[:,c]),2), round(np.std(tbd[:,c]),2)]
    print(ch_str[c]+': ', outs[c,:],'@ ', np.mean(tb_obs[:,c])) #, np.mean(tbd[low[0],c]))
    #print(c,np.min(tbd[:,c]), np.max(tbd[:,c]), np.mean(tbd[:,c]), np.std(tbd[:,c]))
    his = np.histogram(tbd[:,c], bins=hx)
    plt.plot(his[1][:-1], his[0], label=ch_str[c]+':   '+str(outs[c,0])+', '+str(outs[c,1]))
plt.legend(fontsize=14)
plt.savefig('img/AMSR2cal_'+dorb+'_'+str(chcut)+'chi_'+str(lwcut)+'lwp_'+err+'_'+tez+'.png', \
 bbox_inches="tight",transparent=True, dpi=250)
plt.show()
#andstophere

In [ ]:
# save numpy arrays from fwd model simulations
chcut, lwcut = 0.5, 10
np.save('data/tbobs_'+str(chcut)+'chi_'+str(lwcut)+'lwp_'+dorb+'_'+err+'_'+tez+'.npy', tb_obs) # points x nch
np.save('data/tbsim_'+str(chcut)+'chi_'+str(lwcut)+'lwp_'+dorb+'_'+err+'_'+tez+'.npy', tb_sim)
np.save('data/sst_'+str(chcut)+'chi_'+str(lwcut)+'lwp_'+dorb+'_'+err+'_'+tez+'.npy', ss_sav)
np.save('data/tpw_'+str(chcut)+'chi_'+str(lwcut)+'lwp_'+dorb+'_'+err+'_'+tez+'.npy', tp_sav)
np.save('data/wind_'+str(chcut)+'chi_'+str(lwcut)+'lwp_'+dorb+'_'+err+'_'+tez+'.npy', wi_sav)

In [ ]:
# read in from numpy arrays:
# ['0.00E+2', '2.84E+2', '1.74E+2', '5.84E+1', (88512, 14)]
chcut, lwcut = 0.5, 10
tbo_in = np.load('data/tbobs_'+str(chcut)+'chi_'+str(lwcut)+'lwp_'+dorb+'_'+err+'_'+tez+'.npy') # points x nch
tbs_in = np.load('data/tbsim_'+str(chcut)+'chi_'+str(lwcut)+'lwp_'+dorb+'_'+err+'_'+tez+'.npy')
ss_in = np.load('data/sst_'+str(chcut)+'chi_'+str(lwcut)+'lwp_'+dorb+'_'+err+'_'+tez+'.npy')
tp_in = np.load('data/tpw_'+str(chcut)+'chi_'+str(lwcut)+'lwp_'+dorb+'_'+err+'_'+tez+'.npy')
wi_in = np.load('data/wind_'+str(chcut)+'chi_'+str(lwcut)+'lwp_'+dorb+'_'+err+'_'+tez+'.npy')
tbd_in = tbo_in-tbs_in

In [ ]:
dexl = np.where(tp_in<15)[0]
dexm = np.logical_and(tp_in>15,tp_in<40)
dexh = np.where(tp_in>40)[0]
print(np.shape(dexl),np.shape(tp_in[dexm]),np.shape(dexh))
for c in range(nch):
    print(c, np.mean(tbd_in[:,c]), np.mean(tbd_in[dexl,c]), np.mean(tbd_in[dexm,c]), np.mean(tbd_in[dexh,c]))

In [ ]:
print(info(tbd[:,2]))
sev = tbd[:,2]
sevo= tb_obs[:,2]
np.shape(sev)
print(info(tb_obs[:,4]))

In [ ]:
# output scatter plots (maybe with best fit lines?) of bias wrt Tb of scene:
for c in range(nch)[:-6]:
    p = plt.figure(figsize=[10,5])
    plt.scatter(tb_obs[:,c], tbd[:,c], s=2) #, title=ch_str)
    plt.title(ch_str[c])

In [ ]:
# output scatter plots (maybe with best fit lines?) of bias wrt Tb of scene:
for c in range(nch)[:-6]:
    p = plt.figure(figsize=[10,5])
    plt.scatter(sc_sav[:], tbd[:,c], s=2) #, title=ch_str)
    plt.title(ch_str[c])